<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/STT_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 49.1 MB/s eta 0:00:00


In [5]:
!pip install pydub

In [21]:
import speech_recognition as sr
import pandas as pd
from tqdm import tqdm
import re
from pydub import AudioSegment
from pydub.silence import split_on_silence
import matplotlib.pyplot as plt
from pydub.silence import detect_silence

# DATA

In [28]:
data_emotion = pd.read_csv('data_emotion.csv')
data_main = pd.read_csv('data_main.csv')

In [10]:
data_emotion

,Emotion,Path
0,sad,/content/drive/MyDrive/project/data_aihub/5/5e...
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
2,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
3,sad,/content/drive/MyDrive/project/data_aihub/5/5f...
4,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
...,...,...
12674,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12675,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12676,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12677,happy,/content/drive/MyDrive/project/data_aihub/5/5e...


In [29]:
data_main

,Path
0,/content/drive/MyDrive/project/aihub_data_main...
1,/content/drive/MyDrive/project/aihub_data_main...
2,/content/drive/MyDrive/project/aihub_data_main...
3,/content/drive/MyDrive/project/aihub_data_main...
4,/content/drive/MyDrive/project/aihub_data_main...
...,...
409,/content/drive/MyDrive/project/aihub_data_main...
410,/content/drive/MyDrive/project/aihub_data_main...
411,/content/drive/MyDrive/project/aihub_data_main...
412,/content/drive/MyDrive/project/aihub_data_main...


# STT

## emotion

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
def speech_to_text(file_path):
  try:
    with sr.AudioFile(file_path) as source:
      audio = recognizer.record(source)
      text = recognizer.recognize_google(audio, language = "ko-KR")
      return text
  except sr.UnknownValueError:
    return "음성을 인식할 수 없음"
  except sr.RequestError as e:
    return f"STT API 오류: {e}"

In [ ]:
data_emotion['Text'] = [speech_to_text(path) for path in tqdm(data_emotion['Path'])]

100%|██████████| 12679/12679 [3:11:02<00:00,  1.11it/s]


In [ ]:
print(data_emotion[['Path', 'Text']])

                                                    Path  \
0      /content/drive/MyDrive/project/data_aihub/5/5e...   
1      /content/drive/MyDrive/project/data_aihub/4/5e...   
2      /content/drive/MyDrive/project/data_aihub/5_2/...   
3      /content/drive/MyDrive/project/data_aihub/5/5f...   
4      /content/drive/MyDrive/project/data_aihub/5_2/...   
...                                                  ...   
12674  /content/drive/MyDrive/project/data_aihub/5_2/...   
12675  /content/drive/MyDrive/project/data_aihub/5_2/...   
12676  /content/drive/MyDrive/project/data_aihub/5_2/...   
12677  /content/drive/MyDrive/project/data_aihub/5/5e...   
12678  /content/drive/MyDrive/project/data_aihub/5_2/...   

                                 Text  
0                        고마워 지금 하고 올게  
1                        음성을 인식할 수 없음  
2                            주식을 시작했어  
3      어 그렇게 심한 건 아니고 허리 다쳐서 병원에 입원 중  
4           음악을 들으면 우울한 내 마음이 좀 가라앉으며  
...                               ...  

In [ ]:
data_emotion.to_csv('emotion_text.csv', index = False, encoding = 'utf-8-sig')

## main

### 공백

In [22]:
def get_silence_durations(file_path):
    sound = AudioSegment.from_wav(file_path)

    silence_ranges = detect_silence(sound, min_silence_len = 500, silence_thresh = -40)

    silence_durations = [(end - start) for start, end in silence_ranges]

    return silence_durations

In [26]:
def calculate_silence_stats_with_tqdm(data_paths):
    all_silences = []

    for file_path in tqdm(data_paths, desc="Processing files"):
        silences = get_silence_durations(file_path)
        all_silences.extend(silences)

    if all_silences:
        avg_silence = sum(all_silences) / len(all_silences)
        min_silence = min(all_silences)
        return avg_silence / 1000, min_silence / 1000
    else:
        return 0, 0

In [30]:
average_silence, min_silence = calculate_silence_stats_with_tqdm(data_main['Path'])

Processing files:   2%|▏         | 7/414 [07:23<7:09:49, 63.36s/it]


KeyboardInterrupt: 

In [31]:
plt.figure(figsize=(10, 6))
plt.hist(all_silences, bins=20, color='blue', alpha=0.7, edgecolor='black')
plt.axvline(average_silence, color='red', linestyle='dashed', linewidth=1, label=f'Average Silence: {average_silence:.2f}s')
plt.axvline(min_silence, color='green', linestyle='dashed', linewidth=1, label=f'Min Silence: {min_silence:.2f}s')
plt.title('Distribution of Silence Durations')
plt.xlabel('Silence Duration (Seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

average_silence, min_silence

NameError: name 'all_silences' is not defined

<Figure size 1000x600 with 0 Axes>

### STT - 50개씩 나눠서

In [12]:
recognizer = sr.Recognizer()

In [13]:
def split_audio_on_silence(file_path):
  sound = AudioSegment.from_wav(file_path)

  chunks = split_on_silence(sound, min_silence_len = 500, silence_thresh = -40)
  return chunks

In [15]:
def speech_to_text(audio_chunk):
    try:
        audio_data = audio_chunk.export(format="wav")
        with sr.AudioFile(audio_data) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [16]:
def process_audio(file_path):
    chunks = split_audio_on_silence(file_path)
    text_results = []

    for i, chunk in enumerate(chunks):
        text = speech_to_text(chunk)
        text_results.append(f"Chunk {i+1}: {text}")

    return text_results

In [17]:
data_list = []
for file_path in tqdm(data_main['Path']):
    stt_results = process_audio(file_path)
    for result in stt_results:
        data_list.append({'Path': file_path, 'STT 결과': result})

 12%|█▏        | 49/414 [1:43:07<12:48:09, 126.27s/it]


KeyboardInterrupt: 

In [18]:
data_main = pd.DataFrame(data_list)

In [19]:
data_main.to_csv('main_text.csv', index = False, encoding = 'utf-8-sig')